In [ ]:
from datasets import (
  load_dataset,
  Dataset
)
import pandas as pd

In [2]:
DATASET_HUB = "atlasia/AL-Atlas-Moroccan-Darija-Pretraining-Dataset"

In [12]:
dataset_in_hub = load_dataset(DATASET_HUB)

# Load the datasets to combine

In [3]:
jasper_dataset = load_dataset("JasperV13/Darija_Dataset")

Generating train split: 100%|██████████| 2731313/2731313 [00:07<00:00, 367983.00 examples/s]


In [4]:
jasper_darija_instruct_dataset = load_dataset("JasperV13/Darija_instruct")

Generating train split: 100%|██████████| 843/843 [00:00<00:00, 174503.91 examples/s]


In [ ]:
abdeljalil_darija_topic_ds = load_dataset("abdeljalilELmajjodi/darija_topic_ds")

In [ ]:
abdeljalil_darija_s2s_ds = load_dataset("abdeljalilELmajjodi/darija_s2s_ds")

In [ ]:
abdeljalil_darija_qa_ds = load_dataset("abdeljalilELmajjodi/darija_qa_ds")

In [ ]:
abdeljalil_darija_classification_ds = load_dataset("abdeljalilELmajjodi/darija_classification_ds")

In [ ]:
bourbouh_subtitles_dataset = load_dataset("bourbouh/moroccan-darija-youtube-subtitles")

In [ ]:
Darija_QA_dataset = load_dataset("Lyte/Darija-QA")

In [ ]:
mo_darija_merged_dataset = load_dataset("tachicart/mo_darija_merged")

In [6]:
fine_web_sawalni_filtered = load_dataset("sawalni-ai/fw-darija")

In [4]:
atlasia_facebook_darija_dataset = load_dataset("atlasia/facebook_darija_dataset")

Generating train split: 100%|██████████| 1374/1374 [00:00<00:00, 117302.89 examples/s]


In [5]:
atlasia_facebook_darija_dataset

DatasetDict({
    train: Dataset({
        features: ['pageName', 'text', '__index_level_0__'],
        num_rows: 1374
    })
})

In [7]:
atlasia_facebook_darija_dataset = atlasia_facebook_darija_dataset.remove_columns(["__index_level_0__"])
atlasia_facebook_darija_dataset

DatasetDict({
    train: Dataset({
        features: ['pageName', 'text'],
        num_rows: 1374
    })
})

#### MAC dataset available as CSV

In [ ]:
mac_df = pd.read_csv("MAC corpus.csv") # https://github.com/Lafifi-24/arabic-dialect-identification

In [ ]:
mac_df.head()

In [ ]:
# keep only the dialect
filtered_mac_df = mac_df[mac_df['class'] == 'dialectal']

In [ ]:
filtered_mac_df.head()

In [ ]:
mac_dataset = Dataset.from_pandas(filtered_mac_df)

In [ ]:
mac_dataset = mac_dataset.train_test_split(test_size=0.01).remove_columns('__index_level_0__')  # leave 1% for test

In [ ]:
mac_dataset

In [4]:
fineweb_2_ary = load_dataset("Omartificial-Intelligence-Space/FineWeb2-Moroccan-Arabic")

Generating test split: 100%|██████████| 404456/404456 [00:00<00:00, 698677.54 examples/s]


In [5]:
fineweb_2_ary

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 69181074
    })
    test: Dataset({
        features: ['text'],
        num_rows: 404456
    })
})

In [4]:
salaheddine_dataset = load_dataset("atlasia/moroccan_darija_domain_classifier_dataset")

Generating train split: 100%|██████████| 188848/188848 [00:00<00:00, 1574444.92 examples/s]


In [ ]:
social_media_default = load_dataset("atlasia/Social_Media_Darija_DS", "default")
social_media_facebook_comments = load_dataset("atlasia/Social_Media_Darija_DS", "facebook_comments")
social_media_facebook_uncleand_posts = load_dataset("atlasia/Social_Media_Darija_DS", "facebook_uncleand_posts")
wikipedia_audio = load_dataset("atlasia/Moroccan-Darija-Wiki-Audio-Dataset")
wikipedia_text = load_dataset("atlasia/Moroccan-Darija-Wiki-Dataset")

Generating train split: 100%|██████████| 10044/10044 [00:00<00:00, 157278.18 examples/s]


In [50]:
data_translation = load_dataset("BounharAbdelaziz/Terjman-v2-English-Darija-Dataset")

Generating test split: 100%|██████████| 850/850 [00:00<00:00, 157820.20 examples/s]


# Concatenate the datasets

In [21]:
# ------------------------------------------------------------------------------------------------------------ #
# ------------------------------------------------------------------------------------------------------------ #

def extract_and_stack_dataset(dataset, dataset_source, dataset_source_column, use_source_from_dataset, metadata_columns):
    """
    Stack multiple text columns into a single "text" column, keeping metadata, and add dataset_source as a separate column.

    Args:
        dataset (DatasetDict or Dataset): The dataset or dataset dict to process.
        dataset_source (str): The source of the dataset to add as a separate column.
        dataset_source_column (str): In case the dataset has the source, use it.
        use_source_from_dataset (bool): Whether to use the 'source' column from the dataset.
        metadata_columns (list or str): List of columns (or single column) to exclude (used as metadata).

    Returns:
        DatasetDict: A DatasetDict with stacked text, metadata columns, and dataset_source as a separate column.
    """
    if isinstance(metadata_columns, str):
        metadata_columns = [metadata_columns]  # Handle single column as a list

    def transform(sample):
        # Check for columns that don't exist in the dataset
        for col in metadata_columns:
            if col not in sample:
                raise KeyError(f"Column '{col}' is not in the dataset. Available columns: {list(sample.keys())}")

        # Identify text columns to stack (all columns except excluded)
        text_columns = [col for col in sample.keys() if col not in metadata_columns]
        stacked_text = []
        metadata_list = {col: [] for col in metadata_columns}  # Initialize metadata dict

        # Stack text and metadata
        for col in text_columns:
            if isinstance(sample[col], list):  # Handle batch mapping with lists
                stacked_text.extend(sample[col])
                for meta_col in metadata_columns:
                    metadata_list[meta_col].extend(sample[meta_col])
            else:
                stacked_text.append(sample[col])
                for meta_col in metadata_columns:
                    metadata_list[meta_col].append(sample[meta_col])

        # Prepare metadata as string for each entry
        merged_metadata = [
            {meta_col: str(metadata_list[meta_col][i]) for meta_col in metadata_columns}
            for i in range(len(stacked_text))
        ]
        
        # Convert entire metadata to string (by converting the whole dict to a string)
        merged_metadata_as_str = [str(metadata) for metadata in merged_metadata]
       
        # Determine the dataset_source_column value based on whether to use it from the dataset
        if use_source_from_dataset and dataset_source_column in sample:
            if isinstance(sample[dataset_source_column], list):  # Batch processing
                dataset_source_column_value = sample[dataset_source_column]
            else:
                dataset_source_column_value = [sample[dataset_source_column]] * len(stacked_text)
        else:
            dataset_source_column_value = [dataset_source] * len(stacked_text)

            
        return {
            "text": stacked_text,
            "dataset_source": dataset_source_column_value,  # Separate column for dataset source
            "metadata": merged_metadata_as_str,  # Entire metadata as string
        }

    # Apply transformation across splits, preserving split structure
    if isinstance(dataset, DatasetDict):
        new_splits = {}
        for split in dataset.keys():
            print(f"Processing split: {split}...")
            new_splits[split] = dataset[split].map(
                transform,
                batched=True,  # Needed to handle stacking correctly
                remove_columns=dataset[split].column_names
            ).flatten_indices()
        return DatasetDict(new_splits)
    else:
        return dataset.map(
            transform,
            batched=True,
            remove_columns=dataset.column_names
        ).flatten_indices()


# ------------------------------------------------------------------------------------------------------------ #
# ------------------------------------------------------------------------------------------------------------ #

from datasets import DatasetDict, concatenate_datasets

def concat_datasetdicts(d1: DatasetDict, d2: DatasetDict) -> DatasetDict:
    """
    Concatenates two DatasetDict objects, handling missing splits and columns.
    If a split exists in one DatasetDict but not the other, it is included as-is.
    If columns are missing in one dataset, they are added with None values.

    Args:
        d1 (DatasetDict): The first DatasetDict to concatenate.
        d2 (DatasetDict): The second DatasetDict to concatenate.

    Returns:
        DatasetDict: A new DatasetDict with concatenated splits.
    """
    # Get all unique splits from both DatasetDicts
    all_splits = set(d1.keys()).union(d2.keys())

    # Create a new DatasetDict by combining splits
    concatenated = DatasetDict()
    
    for split in all_splits:
        if split in d1 and split in d2:
            # Both DatasetDicts have this split, so concatenate them
            # Get the feature sets from both datasets
            features_d1 = set(d1[split].column_names)
            features_d2 = set(d2[split].column_names)

            # Find missing columns in both datasets
            missing_in_d1 = features_d2 - features_d1
            missing_in_d2 = features_d1 - features_d2

            # Add missing columns with None values
            for missing_col in missing_in_d1:
                d1[split] = d1[split].add_column(missing_col, [None] * len(d1[split]))
            for missing_col in missing_in_d2:
                d2[split] = d2[split].add_column(missing_col, [None] * len(d2[split]))

            # Now concatenate the datasets with the same features
            concatenated[split] = concatenate_datasets([d1[split], d2[split]])

        elif split in d1:
            # Only the first DatasetDict has this split
            concatenated[split] = d1[split]
        elif split in d2:
            # Only the second DatasetDict has this split
            concatenated[split] = d2[split]

    return concatenated

# ------------------------------------------------------------------------------------------------------------ #
# ------------------------------------------------------------------------------------------------------------ #

def combine_all_datasets(all_datasets_dict: dict={}):
    """ Combines all dataset from a given dictionary of structure. """
    
    # Create a new DatasetDict by combining splits
    combined = DatasetDict()
        
    for dataset_source, data in all_datasets_dict.items():
        
        if data["dataset_source_column_name"] is not None:
            use_source_from_dataset = True
            dataset_source_column = data["dataset_source_column_name"]
        else:
            use_source_from_dataset = False
            dataset_source_column = None
        
        dataset = extract_and_stack_dataset(dataset=data["dataset"], 
                                                dataset_source=dataset_source, 
                                                dataset_source_column=dataset_source_column, 
                                                use_source_from_dataset=use_source_from_dataset, 
                                                metadata_columns=data["metadata_columns"]
                )
        combined = concat_datasetdicts(combined, dataset)
        
        print(f"=" * 25)
        
    return combined
    
# ------------------------------------------------------------------------------------------------------------ #
# ------------------------------------------------------------------------------------------------------------ #

In [ ]:
all_datasets_dict= {
                "JasperV13/Darija_Dataset": {
                    "dataset": jasper_dataset,
                    "metadata_columns": ['source'],
                    "dataset_source_column_name": 'source',
                },
                "abdeljalilELmajjodi/darija_topic_ds": {
                    "dataset": abdeljalil_darija_topic_ds,
                    "metadata_columns": ['topic'],
                    "dataset_source_column_name": None,
                },
                "abdeljalilELmajjodi/darija_qa_ds": {
                    "dataset": abdeljalil_darija_qa_ds,
                    "metadata_columns": ['question_number', 'correct_answer_num'],
                    "dataset_source_column_name": None,
                },
                "abdeljalilELmajjodi/abdeljalil_darija_classification_ds": {
                    "dataset": abdeljalil_darija_classification_ds,
                    "metadata_columns": ['index_id', 'category'],
                    "dataset_source_column_name": None,
                },
                "bourbouh/moroccan-darija-youtube-subtitles": {
                    "dataset": bourbouh_subtitles_dataset,
                    "metadata_columns": ['video_id', 'title'],
                    "dataset_source_column_name": None,
                },
                "JasperV13/Darija_instruct": {
                    "dataset": jasper_darija_instruct_dataset,
                    "metadata_columns": [],
                    "dataset_source_column_name": None,
                },
                "tachicart/mo_darija_merged": {
                    "dataset": mo_darija_merged_dataset,
                    "metadata_columns": ['ar'],
                    "dataset_source_column_name": None,
                },
                "Lyte/Darija-QA": {
                    "dataset": Darija_QA_dataset,
                    "metadata_columns": [],
                    "dataset_source_column_name": None,
                },
                "MAC Corpus": {
                    "dataset": mac_dataset,
                    "metadata_columns": ['type', 'class'],
                    "dataset_source_column_name": None,
                },
                "sawalni-ai/fw-darija": {
                    "dataset": fine_web_sawalni_filtered,
                    "metadata_columns": ['gherbal_cleaned_text', 'gherbal_predictions', 'gherbal_lang', 'gherbal_score', 'id', 'metadata', 'domain'],
                    "dataset_source_column_name": None,
                },
            }

In [51]:
data_translation

DatasetDict({
    train: Dataset({
        features: ['english', 'darija_Arab', 'darija_Latn', 'dataset_source', 'id', 'role', 'darija_tokens', 'subtopic', 'topic', 'annotator_dialect'],
        num_rows: 577974
    })
    test: Dataset({
        features: ['english', 'darija_Arab', 'darija_Latn', 'dataset_source', 'id', 'role', 'darija_tokens', 'subtopic', 'topic', 'annotator_dialect'],
        num_rows: 850
    })
})

In [ ]:
to_add_datasets_dict= {
                "BounharAbdelaziz/Terjman-v2-English-Darija-Dataset": {
                    "dataset": data_translation,
                    "metadata_columns": ['english', 'darija_Latn', 'dataset_source', 'id', 'role', 'darija_tokens', 'subtopic', 'topic', 'annotator_dialect'],
                    "dataset_source_column_name": 'dataset_source', # or None if you want to use the key as source
                },
            }

In [53]:
to_add_dataset = combine_all_datasets(to_add_datasets_dict)

Processing split: train...


Flattening the indices: 100%|██████████| 577974/577974 [00:02<00:00, 258919.21 examples/s] 


Processing split: test...


Flattening the indices: 100%|██████████| 850/850 [00:00<00:00, 236746.03 examples/s]

In [54]:
to_add_dataset

DatasetDict({
    test: Dataset({
        features: ['dataset_source', 'text', 'metadata'],
        num_rows: 850
    })
    train: Dataset({
        features: ['dataset_source', 'text', 'metadata'],
        num_rows: 577974
    })
})

In [64]:
combined = concat_datasetdicts(to_add_dataset, dataset_in_hub)

In [66]:
set(combined['train']['dataset_source'])

{'ArabicDarija_xP3x',
 'Darija-Stories-Dataset',
 'DarijaBridge',
 'DarijaEnglish-xP3x',
 'JasperV13/Darija_instruct',
 'Lyte/Darija-QA',
 'MAC Corpus',
 'MArSum',
 'ML101',
 'MTCD',
 'NQ-Open-AYA',
 'SoftAge-AI/sft-conversational_dataset',
 'abdeljalilELmajjodi/abdeljalil_darija_classification_ds',
 'abdeljalilELmajjodi/darija_qa_ds',
 'abdeljalilELmajjodi/darija_topic_ds',
 'aboutaleb/moroccantourism',
 'atlasia/DODa-audio-dataset-V3',
 'atlasia/Moroccan-Darija-Wiki-Audio-Dataset',
 'atlasia/Moroccan-Darija-Wiki-Dataset',
 'atlasia/Social_Media_Darija_DS/default',
 'atlasia/Social_Media_Darija_DS/facebook_comments',
 'atlasia/Social_Media_Darija_DS/facebook_uncleand_posts',
 'atlasia/darija_english',
 'atlasia/darija_english/stories',
 'atlasia/facebook_darija_dataset',
 'atlasia/moroccan_darija_domain_classifier_dataset',
 'auto-math',
 'bourbouh/moroccan-darija-youtube-subtitles',
 'common-crawl',
 'cot',
 'darija_speech_to_text',
 'darija_youtube_subtitles',
 'dataset_dyal_darija'

In [68]:
combined

DatasetDict({
    train: Dataset({
        features: ['dataset_source', 'text', 'metadata', 'token_count'],
        num_rows: 1183419
    })
    test: Dataset({
        features: ['dataset_source', 'text', 'metadata', 'token_count'],
        num_rows: 2706
    })
})

## remove duplicates

In [69]:
# Function to deduplicate based on a column
def deduplicate(dataset, column):
    # Get unique values in the specified column
    unique_values = set(dataset.unique(column))
    
    # Filter dataset to keep only unique rows
    return dataset.filter(lambda example: example[column] in unique_values and not unique_values.remove(example[column]))

In [70]:
# Apply deduplication to each split
combined["train"] = deduplicate(combined["train"], column="text")
combined["test"] = deduplicate(combined["test"], column="text")

# Print sizes after deduplication
print(f"Train size after deduplication: {len(combined['train'])}")
print(f"Test size after deduplication: {len(combined['test'])}")

Filter: 100%|██████████| 2706/2706 [00:00<00:00, 225740.61 examples/s]

Train size after deduplication: 1171515
Test size after deduplication: 2683


In [72]:
combined

DatasetDict({
    train: Dataset({
        features: ['dataset_source', 'text', 'metadata', 'token_count'],
        num_rows: 1171515
    })
    test: Dataset({
        features: ['dataset_source', 'text', 'metadata', 'token_count'],
        num_rows: 2683
    })
})

## count the number of tokens

In [73]:
from transformers import AutoTokenizer
import os

In [75]:
model_path = "core42/jais-13b"
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [76]:
# Function to count tokens in each text entry
def count_tokens(batch):
    # Tokenize the text and count the number of tokens for each entry
    batch["token_count"] = [len(tokenizer.encode(text, truncation=True)) for text in batch["text"]]
    return batch

# Apply the token counting function to both splits
combined["train"] = combined["train"].map(count_tokens, batched=True, batch_size=os.cpu_count())
combined["test"] = combined["test"].map(count_tokens, batched=True, batch_size=os.cpu_count())

Map: 100%|██████████| 2683/2683 [00:00<00:00, 22381.66 examples/s]


In [43]:
# Print some examples to verify token counts
print(combined["train"][0])  # First example from train split with token count
print(combined["test"][0])   # First example from test split with token count

# Optional: Aggregate token counts for each split
total_train_tokens = sum(combined["train"]["token_count"])
total_test_tokens = sum(combined["test"]["token_count"])

print(f"Total tokens in train split: {total_train_tokens}")
print(f"Total tokens in test split: {total_test_tokens}")

{'text': 'الحلقة رقم 63# من البودكاست.\n\nفهاد الحلقة من البودكاست هضرت مع الدكتور اسماعيل سعدون لي متخصص فبطاريات الليثيوم. الدكتور سعدون عندو تجربة واعرة بزاف و خبرة واسعة فهاد المجال. عاود لنا فهاد البودكاست على كيفاش حتى قرا فالجامعة فالمغرب مع الصعوبات اللوجيتسكية و المادية لي كان كيعاني منها. قدر أنه يكون من بين أوائل الدفعة ديالهم و مشا كمل الدكوراه ديالو ففرنسا. من تما رجع للمغرب و قدر يأسس مختبر Laboratory of Materials and Environmental Chemistry فجامعة القاضي عياض. خدم مع عدد ديال الباحثين و قدر يجيب بزاف ديال التمويلات الخارجية باش يدعم المختبر ديالو. ليوما الدكتور سعدون من أكثر الناس خبرة فمجال بطاريات الليثيوم فالعالم. كيعطي محاضرات فأوروبا و آسيا. ألف أكثر من 150 ورقة بحثية و أكثر من 3600 اقتباس و حصل على جائزة من جامعة طوكيو مؤخرا. فهاد الحلقة هضرنا على المسار ديالو و كيفاش حتى حقق هادشي كامل. إضافة لهادشي هضرنا على البحث العلمي فالمغرب و المشاكل لي كاينة و الحلول ديالها. الدكتور سعدون عندو خبرة طويلة فالبحث العلمي فالمغرب، و هو ليوما مدير المحتبر لي ساهم فالتأسيس ديالو.

In [77]:
df_train = combined['train'].to_pandas()
df_test = combined['test'].to_pandas()

In [ ]:
df_train['dataset_source'].value_counts()

1171515

In [ ]:
df_test['dataset_source'].value_counts()

dataset_source
tachicart/mo_darija_merged    1799
                               830
MAC Corpus                      54
Name: count, dtype: int64

In [82]:
df_test['dataset_source'].isna().sum()

0

In [83]:
df_test['dataset_source'] = df_test['dataset_source'].apply(lambda x: 'atlasia/TerjamaBench' if x == '' else x)

In [84]:
df_test['dataset_source'].value_counts()

dataset_source
tachicart/mo_darija_merged    1799
atlasia/TerjamaBench           830
MAC Corpus                      54
Name: count, dtype: int64

# Push the dataset to the Hugging Face Hub

In [80]:
combined.push_to_hub(DATASET_HUB, commit_message="Added Abdeljalil's social media and wikipedia datasets + Removed duplicates + Added token counts.")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.52it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/atlasia/AL-Atlas-Moroccan-Darija-Pretraining-Dataset/commit/a29f4afcebc5eddb3b4da7f3ca8d8de196ba971e', commit_message="Added Abdeljalil's social media and wikipedia datasets + Removed duplicates + Added token counts.", commit_description='', oid='a29f4afcebc5eddb3b4da7f3ca8d8de196ba971e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/atlasia/AL-Atlas-Moroccan-Darija-Pretraining-Dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='atlasia/AL-Atlas-Moroccan-Darija-Pretraining-Dataset'), pr_revision=None, pr_num=None)

# MinHash Deduplication

In [16]:
from datasets import DatasetDict
from datasketch import MinHash, MinHashLSH
from sklearn.feature_extraction.text import CountVectorizer

In [85]:
def run_minhash_deduplication(dataset_dict, text_column, n=3, num_perm=128, threshold=0.8, LOG_FREQUENCY=1000):

    def generate_ngrams(text, n):
        """Generate n-grams from text."""
        vectorizer = CountVectorizer(ngram_range=(n, n), analyzer='word', token_pattern=r'\b\w+\b')
        try:
            vectorizer.fit([text])
            return vectorizer.get_feature_names_out()
        except ValueError:
            return []

    def get_minhash(ngrams, num_perm):
        """Create a MinHash signature from n-grams."""
        m = MinHash(num_perm=num_perm)
        for ngram in ngrams:
            m.update(ngram.encode('utf8'))
        return m

    # Initialize LSH
    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)

    def process_example(example, idx, split_name):
        """Process a single example to check for duplicates."""
        global_id = f"{split_name}_{idx}"
        text = example.get(text_column, None)

        # # Log example details for debugging
        # if idx % LOG_FREQUENCY == 0:  # Log every 1000 examples
        #     print(f"Processing {global_id}: {text[:50] if text else 'EMPTY'}")

        if text is None or not text.strip():
            return {"_keep": False, **example}

        ngrams = generate_ngrams(text, n)
        if len(ngrams) == 0:  # Explicitly check if ngrams array is empty
            return {"_keep": False, **example}

        m = get_minhash(ngrams, num_perm)

        # Check for duplicates
        if lsh.query(m):
            return {"_keep": False, **example}
        else:
            lsh.insert(global_id, m)
            return {"_keep": True, **example}


    # Deduplicate each split
    deduplicated_splits = {}
    for split_name, split_data in dataset_dict.items():
        print(f"Processing split: {split_name} (Size: {len(split_data)})")
        deduplicated_split = split_data.map(
            lambda example, idx: process_example(example, idx, split_name),
            with_indices=True,
            remove_columns=[]  # Keeps the original schema
        ).filter(lambda x: x["_keep"])

        deduplicated_split = deduplicated_split.remove_columns(["_keep"])
        deduplicated_splits[split_name] = deduplicated_split

    print("[INFO] Deduplication complete.")
    return DatasetDict(deduplicated_splits)


In [87]:
LSH_THRESHOLD = 0.75
NUM_PERM = 128
N_GRAM = 3
text_column ='text'
LOG_FREQUENCY = 1000

In [ ]:
# execute minhash dedpulication
deduplicated_dataset = run_minhash_deduplication(dataset_dict=dataset_in_hub, 
                                                text_column=text_column, 
                                                 n=N_GRAM, 
                                                 num_perm=NUM_PERM, 
                                                 threshold=LSH_THRESHOLD,
                                                 LOG_FREQUENCY=LOG_FREQUENCY)